# AFF - Generating Treated Landscape Scenarios
## Requirements:
* LANDFIRE DIST CODE to give all treated pixels
* Total area of your AOI, 
* A structured dictionary of parameters: 
1) target percentage of AOI treated 
2) the sizes of distinct treatment units 
3) ratio of treated area target to be covered by each treatment unit size class
## Outputs an Earth Engine imageCollection comprising one Earth Engine Image per scenario, with treated pixels given a pre-determined LANDFIRE DIST code

In [1]:
DIST_CODE = 222
total_area = 245278 #acres
plan = {
        'scenarios': [0.1,
                     0.3,
                     0.6],
        
        'size_classes': [[60,20,10],
                        [250,100,50],
                        [100,50,10]],
        
        'trt_unit_ratios': [[0.05,0.03,0.02],
                           [0.1,0.1,0.1],
                           [0.3,0.2,0.1]]
       }

In [2]:
import ee
import geemap
ee.Initialize()

parcels = ee.FeatureCollection("projects/aff-treatments/assets/AllParcels_forSIG")
NIparcels = ee.FeatureCollection("projects/aff-treatments/assets/AllNIPParcels_forSIG")
fireshed = ee.FeatureCollection("projects/aff-treatments/assets/SonoraFireshed")
aoi = fireshed

In [3]:
def distanceFilter(pts,distance):
    withinDistance = distance; ##go with pretty far apart at first

    ## From the User Guide: https:#developers.google.com/earth-engine/joins_spatial
    ## add extra filter to eliminate self-matches
    distFilter = ee.Filter.And(ee.Filter.withinDistance(**{
      'distance': withinDistance,
      'leftField': '.geo',
      'rightField': '.geo', 
      'maxError': 1
    }), ee.Filter.notEquals(**{
      'leftField': 'system:index',
      'rightField': 'system:index',

    }));
    
    distSaveAll = ee.Join.saveAll(**{
                  'matchesKey': 'points',
                  'measureKey': 'distance'
    });
    # Apply the join.
    spatialJoined = distSaveAll.apply(pts, pts, distFilter);

    # Check the number of matches.
    # We're only interested if nmatches > 0.
    spatialJoined = spatialJoined.map(lambda f: f.set('nmatches', ee.List(f.get('points')).size()) );
    spatialJoined = spatialJoined.filterMetadata('nmatches', 'greater_than', 0);

    # The real matches are only half the total, because if p1.withinDistance(p2) then p2.withinDistance(p1)
    # Use some iterative logic to clean up
    def unpack(l): 
        return ee.List(l).map(lambda f: ee.Feature(f).id())

    def iterator_f(f,list):
        key = ee.Feature(f).id()
        list = ee.Algorithms.If(ee.List(list).contains(key), list, ee.List(list).cat(unpack(ee.List(f.get('points')))))
        return list
    
    ids = spatialJoined.iterate(iterator_f,ee.List([]))
    ##print("Removal candidates' IDs", ids);

    # Clean up 
    cleaned_pts = pts.filter(ee.Filter.inList('system:index', ids).Not());
    return cleaned_pts



In [4]:
import math
import numpy as np

# function that will do all the math thats needed for a given scenario
def treatment_math(total_area, scenario, size_classes, trt_unit_ratios):
    """Returns the amount of treatment units needed per size class and the radii of the kernel needed for each size class to make the shapes tiered treatment landscape in EE
    # args:
    # total_area (int): total area of aoi to consider treatments in acres
    # scenario (float): percent of total area to be treated in the given scenario, expressed as float [0,1]
    # size_classes (list): list of unique treatment sizes in acres in descending order of size (e.g. [100,50,10] ) 
    # trt_unit_ratios (list): ratio of size classes to make up the given pct treated in the scenario, expressed as a list of floats equaling scenario (i.e. if scenario = 0.6, trt_unit_ratios could be [0.2,0.3,0.1] )
                                must match order of size_classes
    """
    if round(sum(trt_unit_ratios),1) != scenario:
        raise RuntimeError(f"sum of trt_unit_ratios does not equal scenario: ({scenario})")
    if len(size_classes) != len(trt_unit_ratios):
        raise RuntimeError(f"size_classes and trt_unit_ratios must be of same length")
    
    # solve for the radii of the treatment units for each treatment size in meters, radius is passed to an EE function to create the shapes
    acres_to_sqm = [int(round(size_i*4027)) for size_i in size_classes] # convert to sq meters for each size class in acreage
    radii_circles = [int(round(math.sqrt(acreage/math.pi))) for acreage in acres_to_sqm] # solve for r: A = pi(r^2)
    
    # next solve for how many of each treatment units you'd need per size class for each scenario (pct total to treat and the proportions among the size classes)
    area_of_scenario = total_area*scenario # compute total area to be treated per treatment scenario
    area_per_size_class = [int(round(area_of_scenario*ratio)) for ratio in trt_unit_ratios]
    units_per_size_class = [int(round(i)) for i in list(np.divide(area_per_size_class,size_classes))]
    return units_per_size_class, radii_circles

In [13]:
def treatments(units,radii):
    seed = 5155115
    ptsBig = ee.Image.constant(1).clip(aoi).sample(aoi,30,'EPSG:3857',None,units[0]+(units[0]*2),seed,True,4,True) 
    ptsBig = distanceFilter(ptsBig,radii[0]*2.1).limit(units[0])
    #print('Big trt points',ptsBig.size().getInfo())
    areasBig = ptsBig.reduceToImage(['constant'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[0],'meters')).eq(1)

    areasBigmask = areasBig.distance(ee.Kernel.euclidean(radii[0]*2,'meters')).gte(0).Not().unmask(1)

#     Map.addLayer(ptsBig,{},'big pts')
#     Map.addLayer(areasBig, pal, 'big treatments')
#     Map.addLayer(areasBigmask,pal,'areasBigmask')
    
    ptsMedium = areasBigmask.selfMask().sample(aoi,30,'EPSG:3857',None,units[1]+(units[1]*2),seed,True,4,True).map(lambda f: f.set('nd',1))
    ptsMedium = distanceFilter(ptsMedium,radii[1]*2.1).limit(units[1])
    #print('Medium trt points',ptsMedium.size().getInfo())
    areasMedium = ptsMedium.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[1],'meters')).eq(1)

    areasMediummask = areasMedium.distance(ee.Kernel.euclidean(radii[1]*2,'meters')).gte(0).Not().unmask(1)
    blendmask = areasBigmask.add(areasMediummask).lte(1).Not().unmask(1)

#     Map.addLayer(ptsMedium,{},'medium pts')
#     Map.addLayer(areasMedium, pal, 'medium treatments')
#     Map.addLayer(areasMediummask,pal,'areasMedium mask')
#     Map.addLayer(blendmask,pal,'blendmask')
    
    ptsSmall = blendmask.selfMask().sample(aoi,30,'EPSG:3857',None,units[2]+(units[2]*2),seed,True,4,True).map(lambda f: f.set('nd',1))
    ptsSmall = distanceFilter(ptsSmall,radii[2]*2.1).limit(units[2])
    #print('Small trt points',ptsSmall.size().getInfo())
    areasSmall = ptsSmall.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[2],'meters')).eq(1)
    
#     Map.addLayer(ptsSmall, {}, 'small points')
#     Map.addLayer(areasSmall, pal, 'small treatments')
    
    blendTreatments = areasBig.add(areasMedium).add(areasSmall).gte(1).multiply(DIST_CODE).selfMask().rename('DIST')
    
#     Map.addLayer(blendTreatments, pal,'final treatment landscape')

    return ee.Image(blendTreatments)

In [14]:
trt_imgs = []
print(f"Total Area of AOI (ac): {total_area}\n")
for i in list(range(len(plan['scenarios']))):
    print(f"Scenario {i+1}")
    print(f"Target Percent Treated: {plan['scenarios'][i]}")
    print(f"Size Classes (ac): {plan['size_classes'][i]}")
    print(f"Ratios of Percent Treated Target per Size Class : {plan['trt_unit_ratios'][i]}")
    units,radii = treatment_math(total_area=total_area, scenario=plan['scenarios'][i], size_classes = plan['size_classes'][i],trt_unit_ratios= plan['trt_unit_ratios'][i])
    #print(units, radii)
    trt_img = treatments(units,radii).set('scenario',i+1)
    trt_imgs.append(trt_img)
    print('\n')

trt_imgs = ee.ImageCollection.fromImages(ee.List(trt_imgs))

Total Area of AOI (ac): 245278

Scenario 1
Target Percent Treated: 0.1
Size Classes (ac): [60, 20, 10]
Ratios of Percent Treated Target per Size Class : [0.05, 0.03, 0.02]


Scenario 2
Target Percent Treated: 0.3
Size Classes (ac): [250, 100, 50]
Ratios of Percent Treated Target per Size Class : [0.1, 0.1, 0.1]


Scenario 3
Target Percent Treated: 0.6
Size Classes (ac): [100, 50, 10]
Ratios of Percent Treated Target per Size Class : [0.3, 0.2, 0.1]




In [15]:
Map = geemap.Map()

Map.addLayer(parcels,{},'parcels')
Map.addLayer(NIparcels,{},'NIP parcels')
Map.addLayer(aoi,{},'fireshed')
Map.centerObject(aoi,12)

pal = {'min':0,'max':1,'palette':['black','white']}
#print(trt_imgs.first().propertyNames().getInfo())
Map.addLayer(trt_imgs.sort('scenario',False).first(), pal, 'final trt landscapes')

Map

Map(center=[38.06601024718689, -120.26096170747162], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Export 